In [1]:
import argparse
import cv2
import numpy as np
import os
import torch
import torch.nn.functional as F
from torchvision.transforms import Compose
from tqdm import tqdm
import glob

import sys
sys.path.insert(0, "Depth-Anything")

from depth_anything.dpt import DepthAnything
from depth_anything.util.transform import Resize, NormalizeImage, PrepareForNet       

In [2]:
DIR_DATA = "/ssd/Datasets_and_code/nuscenes_depth_estimation/dataset/dataset_radar_cam"
ENCODER = 'vitb'

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

cuda


In [3]:
# Get the current directory
current_dir = os.getcwd()
# Save the original working directory
original_cwd = os.getcwd()
# Change the working directory to Depth-Anything
os.chdir(os.path.join(current_dir, 'Depth-Anything'))
# Load model
depth_anything = DepthAnything.from_pretrained('LiheYoung/depth_anything_{}14'.format(ENCODER)).to(DEVICE).eval()
# Revert back to the original working directory
os.chdir(original_cwd)
total_params = sum(param.numel() for param in depth_anything.parameters())
print('Total parameters: {:.2f}M'.format(total_params / 1e6))

xFormers not available
xFormers not available


Total parameters: 97.47M


In [4]:
transform = Compose([
        Resize(
            width=518,
            height=518,
            resize_target=False,
            keep_aspect_ratio=True,
            ensure_multiple_of=14,
            resize_method='lower_bound',
            image_interpolation_method=cv2.INTER_CUBIC,
        ),
        NormalizeImage(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        PrepareForNet(),
    ])

In [5]:
dir_data_out = os.path.join(DIR_DATA, 'relative_depth')
dir_data_out_png = os.path.join(DIR_DATA, 'relative_depth_png')

if not os.path.exists(dir_data_out):
    os.makedirs(dir_data_out)

if not os.path.exists(dir_data_out_png):
    os.makedirs(dir_data_out_png)
# Remove all files in the output folder
f_list=glob.glob(os.path.join(dir_data_out,'*'))
for f in f_list:
    os.remove(f)
print('removed %d old files in output folder' % len(f_list))

f_list=glob.glob(os.path.join(dir_data_out_png,'*'))
for f in f_list:
    os.remove(f)
print('removed %d old files in output folder png' % len(f_list))

removed 404 old files in output folder
removed 404 old files in output folder png


In [6]:
filename_images = glob.glob(os.path.join(DIR_DATA,'images','*'))
print(len(filename_images))

404


In [ ]:
for i in tqdm(range(len(filename_images))):
    
    raw_image = cv2.imread(filename_images[i])
    image = cv2.cvtColor(raw_image, cv2.COLOR_BGR2RGB) / 255.0

    h, w = image.shape[:2]
        
    image = transform({'image': image})['image']
    image = torch.from_numpy(image).unsqueeze(0).to(DEVICE)
    
    with torch.no_grad():
        depth = depth_anything(image)
    
    depth = F.interpolate(depth[None], (h, w), mode='bilinear', align_corners=False)[0, 0]
    depth = (depth - depth.min()) / (depth.max() - depth.min()) #* 255.0
    
    depth = depth.cpu().numpy() #.astype(np.uint8)

    depth_png = cv2.applyColorMap((depth*255.0).astype(np.uint8), cv2.COLORMAP_INFERNO)

    sample_idx = int(filename_images[i][-12:-7])

    np.save(os.path.join(dir_data_out, '%05d_rel_depth' % sample_idx), depth)
    cv2.imwrite(os.path.join(dir_data_out_png, '%05d_rel_depth.png' % sample_idx), depth_png)

 50%|████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                | 202/404 [01:40<01:35,  2.13it/s]